In [1]:
!pip install google-cloud-pipeline-components==0.1.4
!pip install google-cloud-aiplatform==1.3.0

In [2]:
import kfp
from datetime import datetime
from kfp.v2 import compiler
from kfp.v2.google.client import AIPlatformClient
from google.cloud import aiplatform
from google_cloud_pipeline_components import aiplatform as gcc_aip

In [73]:
PROJECT_ID=!gcloud config get-value project # returns default project id 
PROJECT_ID=PROJECT_ID[0]
TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")
BUCKET = "gs://"+PROJECT_ID+'/telco-churn/pipeline'
REGION='europe-west4'

VERSION='v1'
JOB_NAME="telco-churn-job-"+VERSION
MODEL_NAME="telco-churn-model-"+VERSION
DATASET_ID='5848592619210276864'

In [74]:
gcs_csv_path = "gs://{}/data/telco/Telco-Customer-Churn.csv".format(PROJECT_ID)
bq_data_table= 'bq://{}.telco.churn'.format(PROJECT_ID)
bq_split_table= 'bq://{}.telco.churn_'.format(PROJECT_ID)
pipeline_root_path = BUCKET+"/pl-root"
pipeline_path = pipeline_root_path+'/churn_classif_pipeline.json'

In [87]:
@kfp.dsl.pipeline(name="{}".format(JOB_NAME),
    pipeline_root=pipeline_root_path)
def pipeline(project_id: str, 
             location: str):
        
    dataset_create_op = gcc_aip.TabularDatasetCreateOp(
        project=project_id, 
        display_name="churn-pred", 
        #gcs_source=gcs_csv_path,
        bq_source=bq_data_table,
        location=location
    )
    
    training_op = gcc_aip.CustomPythonPackageTrainingJobRunOp(
        project=project_id,
        display_name="train_churn_prediction_{}".format(VERSION),
        python_package_gcs_uri="gs://{}/telco-churn/dist/trainer-0.1.tar.gz".format(PROJECT_ID),
        python_module="trainer.task",
        model_description="Churn prediction model",
        container_uri='eu.gcr.io/cloud-aiplatform/training/scikit-learn-cpu.0-23:latest', 
        model_serving_container_image_uri='eu.gcr.io/cloud-aiplatform/prediction/sklearn-cpu.0-23:latest',
        dataset=dataset_create_op.outputs["dataset"],
        model_display_name="myChurnModel{}".format(VERSION),
        base_output_dir="{}/assets-{}".format(BUCKET, VERSION),
        staging_bucket="{}/staging".format(BUCKET),
        bigquery_destination="bq://{}".format(PROJECT_ID),
        machine_type= 'n1-standard-4',
        training_fraction_split = 0.8,
        validation_fraction_split = 0.1,
        test_fraction_split = 0.1,
        location=location
    )

    deploy_op = gcc_aip.ModelDeployOp(  
        model=training_op.outputs["model"],
        project=project_id,
        machine_type="n1-standard-4",
        location=location
    )

In [88]:
compiler.Compiler().compile(
    pipeline_func=pipeline,
    package_path='/tmp/churn_classif_pipeline.json')

In [89]:
from google.cloud import storage
storage_client = storage.Client()
bucket = storage_client.bucket(PROJECT_ID)
blob = bucket.blob('/'.join(pipeline_path.split('/')[3:]))

blob.upload_from_filename('/tmp/churn_classif_pipeline.json')

In [90]:
""" 
from kfp.v2.google.client import AIPlatformClient
api_client = AIPlatformClient(project_id=PROJECT_ID, region=REGION)
response = api_client.create_run_from_job_spec(
    'churn_classif_pipeline.json',
    pipeline_root=pipeline_root_path,
    parameter_values={
        'project_id': project_id
    })
    
"""

from google.cloud.aiplatform.pipeline_jobs import PipelineJob

pl = PipelineJob(display_name= JOB_NAME,
        template_path= pipeline_path,
        location=REGION,
        parameter_values={'project_id': PROJECT_ID, 
                          'location': REGION})

pl.run(sync=False)

INFO:google.cloud.aiplatform.pipeline_jobs:Creating PipelineJob
INFO:google.cloud.aiplatform.pipeline_jobs:PipelineJob created. Resource name: projects/478111835512/locations/europe-west4/pipelineJobs/telco-churn-job-v1-20210824000111
INFO:google.cloud.aiplatform.pipeline_jobs:To use this PipelineJob in another session:
INFO:google.cloud.aiplatform.pipeline_jobs:pipeline_job = aiplatform.PipelineJob.get('projects/478111835512/locations/europe-west4/pipelineJobs/telco-churn-job-v1-20210824000111')
INFO:google.cloud.aiplatform.pipeline_jobs:View Pipeline Job:
https://console.cloud.google.com/vertex-ai/locations/europe-west4/pipelines/runs/telco-churn-job-v1-20210824000111?project=478111835512
INFO:google.cloud.aiplatform.pipeline_jobs:PipelineJob projects/478111835512/locations/europe-west4/pipelineJobs/telco-churn-job-v1-20210824000111 current state:
PipelineState.PIPELINE_STATE_RUNNING
INFO:google.cloud.aiplatform.pipeline_jobs:PipelineJob projects/478111835512/locations/europe-west4/p

In [85]:
pl.state

<PipelineState.PIPELINE_STATE_RUNNING: 3>